In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from collections import Counter
import nltk
import psycopg2
from nltk.corpus import stopwords
import re
from itertools import chain 
from IPython.display import Image
import math
import time
import logging 

# Improvement logs
### Bio capture techniques
### normalize() improv


In [2]:
from gensim import models
w = models.KeyedVectors.load_word2vec_format(r"F:\Pretrained Models\GoogleNews-vectors-negative300.bin.gz", binary=True)

In [16]:
def process(array):
    text = re.sub(r'\[[0-9]*\]',' ',str(array))  #Remove Numbers
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Remove nums
    text = re.sub(r'\s+',' ',text)  #Remove extra space
    text = re.sub(r"[^a-zA-Z0-9]+",' ',text)  #Remove special characters
    text = text.lower()  #Lower case all
    text = nltk.sent_tokenize(text)  #Tokenize to sentences 
    keywords = [nltk.word_tokenize(sentence) for sentence in text]
    for i in range(len(keywords)):
        keywords[i] = [word for word in keywords[i] if word not in stopwords.words('english')]
    return keywords

#Frame pre-processing function
def prepro(frame,cap_or_bio):
    frame2 = frame
    for c in range(len(frame2)):
        keywords = process(frame2[cap_or_bio][c])
        frame2[cap_or_bio][c] = keywords
    return frame2

def flat(array):     # 2D array reduction
    flatten_list = [j for sub in array for j in sub] 
    flatten_list = list(chain.from_iterable(flatten_list))
    return flatten_list


# normalize() -> given an array, converts to 1/0, top int(pos) will be 1
def normalize(keys, pos =3):  
    ax = [i for i in keys]
    temp = [i for i in keys]
    temp.sort()
    temp = temp[-pos:]
    for x in temp:
        ax[keys.index(x)] = 1
    for x in range(len(ax)):
        if ax[x] != 1:
            ax[x] = 0
    return ax

# compute() => category[] to be called outside
def compute(caption,category,top =3):
    ar = []
    score = []
    
    # Code to get frequency distribution and unique keywords array
    keywords = []
    caption_freq = []
    counts = Counter(caption)
    if len(counts) > 0:
        labels, values = zip(*counts.items())
        ## sort your values in descending order
        indSort = np.argsort(values)[::-1]
        ## rearrange your data
        keywords = np.array(labels)[indSort]  # Label
        caption_freq = np.array(values)[indSort]  # Values
    
    # Detect words not in Google Dict | Put freq = 0
    for x in keywords:
        try:
            restConst = w.similarity(x,'something')
        except KeyError:
            caption_freq[np.where(keywords == x)] = 0
        
    #Google similaity function
    for x in category:
        empty = []
        for y in keywords:
            try:
                empty.append(w.similarity(x,y))
            except:
                empty.append(0)
        ar.append(empty)
    
    # Store the similarity values in dataframe
    frame = pd.DataFrame(ar, columns = keywords)
    
    #Normalize | top select
    for key in frame.columns:
        frame[key] = normalize(frame[key].tolist(),top)
    
    # Multiply with frequency
    for row in range(len(frame)):
        frame.values[row] = [i*j for i,j in zip(frame.values[row],caption_freq)]
    # Sum the values => Score
    for row in range(len(frame)):
        score.append(sum(frame.values[row]))
    
    frame['category'] = category
    frame['Scores'] = score
    return frame

def deviation(array):
    mu = max(array)
    l = len(array)
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        if total != 0:
            ar[x] = (ar[x]/total)*100
    return ar

def dev_shortlist(dev_array,thre = 2):  # Shortlist using threshold from deviation 
    final_cat = [0]*len(dev_array)
    for i in range(len(dev_array)):
        if dev_array[i] <=thre:
            final_cat[i] = 1
    return final_cat

def sort_cat(dev,cat, thre = 2):   # Shortlist using thre | Return category array
    final_cat = []
    for i in range(len(dev)):
        if dev[i] <=thre:
            final_cat.append(cat[i])
    return final_cat

def get_row_pscore(col_name,f1,f2, scoreType):  # f1-mainframe | f2-frame
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    dev_array = f2[scoreType].tolist()
    row_in_array.extend(dev_array)
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict
def top_category(f2,categories,thre=2):
    final_cat = [0]*len(categories)
    dev_scores = f2['Deviation'].tolist()
    rank_of_cat = 1
    for i in range(len(dev_scores)):
        if dev_scores[i] <=thre:
            final_cat[i] = rank_of_cat 
            rank_of_cat = rank_of_cat +1
    return final_cat

def top_category_get_percent(f2,categories,thre=2):
    final_cat = [0]*len(categories)
    dev_scores = f2['Deviation'].tolist()
    percent_array = f2['Percentage'].tolist()
    for i in range(len(dev_scores)):
        if dev_scores[i] <=thre:
            final_cat[i] = percent_array[i]
    return final_cat
    
    
def get_row_result(col_name,f1,rank_array):  # f1-mainframe
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    row_in_array.extend(rank_array)
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict

In [2]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

In [10]:
cur.execute('''SELECT  "instagram_accounts"."user_id", "instagram_accounts"."handle", "instagram_accounts"."url", "instagram_accounts"."followers" FROM "instagram_accounts" WHERE (followers < 500000 and country = 'India') ORDER BY "instagram_accounts"."followers" DESC''')
InstagramAccount = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
InstagramAccount = InstagramAccount[700:1700]
Insta_ids = InstagramAccount['user_id'].tolist()

In [12]:
# Categories
categories = ['food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']


# DataFrame Creation

profile_deviations = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
profile_percentages = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
onefive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
twofive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
threefive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
twofive_in_percent = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
bio_categorised = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
bio_n_caption_cat = pd.DataFrame(columns = ['user_id','handle','followers','url','categorised by bio','categorised by caption']) 

# This required to run some function

col_name = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']


In [12]:
#Timer start

tic = time.perf_counter()
prev_tac = tic
for x in Insta_ids:
    try:
        tac = time.perf_counter()
        time_compare = tac-prev_tac
        if time_compare >= 600:
#             cur = conn.close()
            conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
            cur = conn.cursor()
            prev_tac = tac
            
        userid = x
        
        # Id, handle, url
        cur.execute('SELECT user_id, handle, url, followers  FROM instagram_accounts WHERE user_id = '+str(userid)+' ORDER BY user_id')
        mainframe = pd.DataFrame()
        mainframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
        # captions
        cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
        randomframe = pd.DataFrame()
        randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
        if len(randomframe) ==0:
            raise Exception("No captions in Winkl database")
        r2 = pd.DataFrame()
        r2 = prepro(randomframe,'Caption')
        # Merge
        r3 = pd.DataFrame()
        r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
        caption_array = r3['Caption'][x]
        # to 1D
        caption_array = flat(caption_array)
        # Word2vec computation
        frame = pd.DataFrame()
        frame = pd.DataFrame(compute(caption_array,categories,3))
        #Convert to Percentage
        per = frame['Scores'].tolist()
        per_sum = sum(per)
        for x in range(len(per)):
            per[x] = round((per[x]/per_sum)*100)
        frame['Percentage'] = per
        # Deviation of categories
        dev = deviation(frame['Scores'].tolist())
        frame['Deviation'] = dev
        
        # store profile deviation
        row_df_1 = get_row_pscore(col_name,mainframe,frame,'Deviation')
        profile_deviations = profile_deviations.append(row_df_1,ignore_index=True)

        # store onefive result
        select_15_array = top_category(frame,categories,thre=1.5)
        row_df_2 = get_row_result(col_name,mainframe,select_15_array)
        onefive = onefive.append(row_df_2,ignore_index=True)

        # store twofive result
        select_25_array = top_category(frame,categories,thre=2.5)
        row_df_3 = get_row_result(col_name,mainframe,select_25_array)
        twofive = twofive.append(row_df_3,ignore_index=True)

        # store threefive result
        select_35_array = top_category(frame,categories,thre=3.5)
        row_df_4 = get_row_result(col_name,mainframe,select_35_array)
        threefive = threefive.append(row_df_4,ignore_index=True)
        
        # Store profile percentage
        row_df_5 = get_row_pscore(col_name,mainframe,frame,'Percentage')
        profile_percentages = profile_percentages.append(row_df_5,ignore_index=True)
        
        # 2.5 categorize with percentage value
        select_25_array_percent = top_category_get_percent(frame,categories,thre=2.5)
        row_df_6 = get_row_result(col_name,mainframe,select_25_array_percent)
        twofive_in_percent = twofive_in_percent.append(row_df_6,ignore_index=True)
        
        # Request Bio & Process it
        cur.execute('SELECT  "instagram_accounts"."user_id","instagram_accounts"."handle", "instagram_accounts"."bio" FROM "instagram_accounts" WHERE (user_id ='+str(user_id)+')')
        biorandomframe = pd.DataFrame(cur.fetchall(),columns=["user_id","handle", "bio"])
        bioprocess = pd.DataFrame()
        bioprocess = prepro(biorandomframe,'bio')
        bio_array = bioprocess['bio'].tolist()
        # [[]] => []
        try:
            bio_array = bio_array[0][0]
        except:
            bio_array = bio_array[0]
        bioframe = pd.DataFrame()
        len(bio_array)
        bioframe = pd.DataFrame(compute(bio_array,categories,3))
        # #Convert to Percentage
        biopercent = bioframe['Scores'].tolist()
        bioper_sum = sum(biopercent)
        if bioper_sum !=0:
            for x in range(len(biopercent)):
                biopercent[x] = round((biopercent[x]/bioper_sum)*100)
        bioframe['Percentage'] = biopercent
        # Deviation of categories
        bio_dev = deviation(bioframe['Scores'].tolist())
        bioframe['Deviation'] = bio_dev
        
        # 2.5 categorize BIO with percent store
        select_25_bio_percent = top_category_get_percent(bioframe,categories,thre=2.5)
        row_df_7 = get_row_result(col_name,mainframe,select_25_bio_percent)
        bio_categorised = bio_categorised.append(row_df_7,ignore_index=True)
        
        # 2.5 BIO + CAPTION string categorised
        bio_cat_scores = bioframe['Scores'].tolist()
        bio_dev_score = deviation(bio_cat_scores)
        bio_row = {'user_id':mainframe.at[0,'user_id'],'handle':mainframe.at[0,'handle'],'followers':mainframe.at[0,'followers'],'url':mainframe.at[0,'url'],'categorised by caption':str(sort_cat(dev,categories,thre = 2.5)),'categorised by bio':str(sort_cat(bio_dev_score,categories,thre = 2.5))}
        bio_n_caption_cat = bio_n_caption_cat.append(bio_row,ignore_index=True)
    except Exception as Argument:
        
        # creating/opening a file 
        f = open(r"E:\Winkl Mains\DATA\googleNews 500 for analysis\errorfile.txt", "a") 
        # writing in the file 
        f.write("In user_id:"+str(userid)+": "+str(Argument)+str("\n")) 
        # closing the file 
        f.close()  

print("profile_deviations")
display(profile_deviations)

print("profile_percentages")
display(profile_percentages)

print("onefive")
display(onefive)

print("twofive")
display(twofive)

print("threefive")
display(threefive)

print("twofive_in_percent")
display(twofive_in_percent)

print("bio_categorised")
display(bio_categorised)

print("bio_n_caption_cat")
display(bio_n_caption_cat)
# Timer end
toc = time.perf_counter()
print(f"Ran the model in {toc - tic:0.4f} seconds")

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


profile_deviations


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,269759,jinatarafder,258550,https://www.instagram.com/jinatarafder/,5.564516,4.354839,0.000000,0.403226,5.161290,5.241935,...,5.806452,5.403226,5.080645,4.677419,10.161290,5.483871,9.919355,10.080645,5.806452,7.580645
1,217672,incredibleindia,258216,https://www.instagram.com/incredibleindia/,6.003776,7.388295,7.438641,0.000000,4.153556,4.858402,...,4.191315,6.683449,7.992448,7.740717,7.979862,5.147892,6.859660,8.319698,6.016362,6.847074
2,244025,saree_the_hottest,257988,https://www.instagram.com/saree_the_hottest,7.647764,0.798722,2.695687,0.000000,5.351438,6.609425,...,7.667732,6.489617,7.767572,5.670927,6.948882,4.233227,7.867412,7.907348,6.869010,7.667732
3,215934,patralekhaa,257908,https://www.instagram.com/patralekhaa/,5.504183,6.076618,4.887715,2.597974,5.680317,8.058124,...,6.957288,7.485689,8.058124,7.617790,10.127697,6.957288,1.937472,9.291061,7.221488,0.000000
4,136985,raool27,257287,https://www.instagram.com/raool27/,5.925698,6.760823,4.071231,3.199263,3.641388,6.656432,...,6.754682,8.455634,5.397605,6.859073,9.143383,5.176543,4.427387,5.575683,0.000000,9.413571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,13205,apoorvajayarajan,177699,https://www.instagram.com/apoorvajayarajan/,4.947651,5.159961,0.239369,0.000000,0.959557,7.345503,...,5.395167,8.107321,0.237287,5.216161,10.652957,7.974107,8.411215,10.163812,7.360073,8.521533
570,217210,sumanmodi_,177694,https://www.instagram.com/sumanmodi_/,6.142035,1.983365,1.855406,0.703775,1.471529,7.485605,...,1.535509,3.518874,0.000000,6.269994,10.428663,1.535509,11.644274,13.371721,7.549584,10.876520
571,263957,theitaofficial,177578,https://www.instagram.com/theitaofficial,10.043668,4.366812,5.240175,0.000000,9.388646,7.641921,...,3.493450,2.401747,7.423581,3.930131,6.986900,1.528384,7.641921,10.043668,1.310044,9.606987
572,249028,jfwdigital,177568,https://www.instagram.com/jfwdigital,4.748908,6.604803,2.893013,5.895197,6.550218,8.897380,...,8.460699,7.096070,8.024017,6.713974,8.788210,6.604803,0.709607,9.552402,7.532751,0.927948


profile_percentages


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,269759,jinatarafder,258550,https://www.instagram.com/jinatarafder/,6,7,11,11,7,6,...,6,6,7,7,2,6,2,2,6,4
1,217672,incredibleindia,258216,https://www.instagram.com/incredibleindia/,6,4,4,13,8,7,...,8,5,3,4,3,7,5,3,6,5
2,244025,saree_the_hottest,257988,https://www.instagram.com/saree_the_hottest,1,20,15,23,7,4,...,1,4,1,6,3,11,0,0,3,1
3,215934,patralekhaa,257908,https://www.instagram.com/patralekhaa/,6,6,7,9,6,4,...,5,4,4,4,2,5,10,2,5,12
4,136985,raool27,257287,https://www.instagram.com/raool27/,6,5,7,8,8,5,...,5,4,6,5,3,6,7,6,10,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,13205,apoorvajayarajan,177699,https://www.instagram.com/apoorvajayarajan/,7,6,10,10,9,5,...,6,4,10,6,3,4,4,3,5,4
570,217210,sumanmodi_,177694,https://www.instagram.com/sumanmodi_/,6,8,8,8,8,5,...,8,7,9,6,4,8,3,2,5,4
571,263957,theitaofficial,177578,https://www.instagram.com/theitaofficial,3,7,6,11,3,4,...,8,9,5,7,5,9,4,3,9,3
572,249028,jfwdigital,177568,https://www.instagram.com/jfwdigital,7,5,9,6,5,3,...,3,5,4,5,3,5,11,2,4,11


...

bio_categorised


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,269759,jinatarafder,258550,https://www.instagram.com/jinatarafder/,0,0,0,0,0,0,...,0,21,0,0,0,17,17,0,0,0
1,217672,incredibleindia,258216,https://www.instagram.com/incredibleindia/,0,0,11,11,0,0,...,0,0,0,0,0,0,11,0,0,0
2,244025,saree_the_hottest,257988,https://www.instagram.com/saree_the_hottest,12,0,10,0,0,0,...,10,12,0,0,0,0,0,0,0,0
3,215934,patralekhaa,257908,https://www.instagram.com/patralekhaa/,0,17,0,17,0,0,...,17,0,0,0,0,0,0,0,17,17
4,136985,raool27,257287,https://www.instagram.com/raool27/,0,0,0,0,0,0,...,0,0,15,11,0,0,0,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,238748,indiangirls.buzz,177784,https://www.instagram.com/indiangirls.buzz,0,0,12,12,0,0,...,0,0,0,0,0,0,0,0,12,0
536,13205,apoorvajayarajan,177699,https://www.instagram.com/apoorvajayarajan/,0,0,0,0,0,0,...,0,0,19,0,0,0,0,0,0,0
537,217210,sumanmodi_,177694,https://www.instagram.com/sumanmodi_/,0,0,0,0,0,0,...,20,17,0,0,0,0,0,0,0,0
538,263957,theitaofficial,177578,https://www.instagram.com/theitaofficial,0,0,0,0,0,0,...,0,0,0,0,0,33,0,0,0,0


bio_n_caption_cat


,user_id,handle,followers,url,categorised by bio,categorised by caption
0,269759,jinatarafder,258550,https://www.instagram.com/jinatarafder/,"['photography', 'entertainment', 'technology']","['makeup', 'beauty']"
1,217672,incredibleindia,258216,https://www.instagram.com/incredibleindia/,"['makeup', 'beauty', 'technology']","['beauty', 'traveler']"
2,244025,saree_the_hottest,257988,https://www.instagram.com/saree_the_hottest,"['food', 'makeup', 'art', 'photography']","['fashion', 'beauty']"
3,215934,patralekhaa,257908,https://www.instagram.com/patralekhaa/,"['fashion', 'beauty', 'art', 'education', 'ani...","['traveler', 'technology', 'animal']"
4,136985,raool27,257287,https://www.instagram.com/raool27/,"['fitness', 'sports', 'education']",['education']
...,...,...,...,...,...,...
535,238748,indiangirls.buzz,177784,https://www.instagram.com/indiangirls.buzz,"['makeup', 'beauty', 'education']",['fashion']
536,13205,apoorvajayarajan,177699,https://www.instagram.com/apoorvajayarajan/,['fitness'],"['makeup', 'beauty', 'lifestyle', 'fitness']"
537,217210,sumanmodi_,177694,https://www.instagram.com/sumanmodi_/,"['art', 'photography']","['fashion', 'makeup', 'beauty', 'lifestyle', '..."
538,263957,theitaofficial,177578,https://www.instagram.com/theitaofficial,['entertainment'],"['beauty', 'photography', 'entertainment', 'ed..."


Ran the model in 5294.5984 seconds


In [13]:
profile_deviations.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_deviations.csv')
profile_percentages.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_percentages.csv')
onefive.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_cat_at_1.5.csv')
twofive.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_cat_at_2.5.csv')
threefive.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_cat_at_3.5.csv')
twofive_in_percent.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_cat_in_percentage.csv')
bio_categorised.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_cat_using_bio.csv')
bio_n_caption_cat.to_csv(r'E:\Winkl Mains\DATA\googleNews 500 for analysis\next_1000\profile_cat_caption_n_bio.csv')

# Bio Improvement code
## Easy code to get bio analysis

In [33]:
ussid = 237098

cur.execute('SELECT  "instagram_accounts"."user_id","instagram_accounts"."handle", "instagram_accounts"."bio" FROM "instagram_accounts" WHERE (user_id ='+str(ussid)+')')
biorandomframe = pd.DataFrame(cur.fetchall(),columns=["user_id","handle", "bio"])
bioprocess = pd.DataFrame()
bioprocess = prepro(biorandomframe,'bio')
bio_array = bioprocess['bio'].tolist()
# [[]] => []
try:
    bio_array = bio_array[0][0]
except:
    bio_array = bio_array[0]
bioframe = pd.DataFrame()
len(bio_array)
bioframe = pd.DataFrame(compute(bio_array,categories,3))
# #Convert to Percentage
biopercent = bioframe['Scores'].tolist()
bioper_sum = sum(biopercent)
if bioper_sum !=0:
    for x in range(len(biopercent)):
        biopercent[x] = round((biopercent[x]/bioper_sum)*100)
bioframe['Percentage'] = biopercent
# Deviation of categories
bio_dev = deviation(bioframe['Scores'].tolist())
bioframe['Deviation'] = bio_dev
display(biorandomframe)
display(bioframe)

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,user_id,handle,bio
0,237098,suman_kashyap345,[]


,category,Scores,Percentage,Deviation
0,food,0,0,0.0
1,fashion,0,0,0.0
2,makeup,0,0,0.0
3,beauty,0,0,0.0
4,lifestyle,0,0,0.0
5,luxury,0,0,0.0
6,traveler,0,0,0.0
7,art,0,0,0.0
8,photography,0,0,0.0
9,fitness,0,0,0.0


# Caption computation modification
## compute() function check
## normalize() function improvement

In [35]:
def normalize_UC(keys, t = 2.5):  
    ax = deviation(keys)
    temp = dev_shortlist(ax,thre =t)
    return temp

def compute_UC(caption,category,top =3):
    ar = []
    score = []
    
    # Code to get frequency distribution and unique keywords array
    keywords = []
    caption_freq = []
    counts = Counter(caption)
    if len(counts) > 0:
        labels, values = zip(*counts.items())
        ## sort your values in descending order
        indSort = np.argsort(values)[::-1]
        ## rearrange your data
        keywords = np.array(labels)[indSort]  # Label
        caption_freq = np.array(values)[indSort]  # Values
    
    # Detect words not in Google Dict | Put freq = 0
    for x in keywords:
        try:
            restConst = w.similarity(x,'something')
        except KeyError:
            caption_freq[np.where(keywords == x)] = 0
        
    #Google similaity function
    for x in category:
        empty = []
        for y in keywords:
            try:
                empty.append(w.similarity(x,y))
            except:
                empty.append(0)
        ar.append(empty)
    
    # Store the similarity values in dataframe
    frame = pd.DataFrame(ar, columns = keywords)
    
#     #Normalize | top select
#     for key in frame.columns:
#         frame[key] = normalize(frame[key].tolist(),top)
    
#     # Multiply with frequency
#     for row in range(len(frame)):
#         frame.values[row] = [i*j for i,j in zip(frame.values[row],caption_freq)]
#     # Sum the values => Score
#     for row in range(len(frame)):
#         score.append(sum(frame.values[row]))
    
#     frame['category'] = category
#     frame['Scores'] = score
    return frame
#### END OF FUNCTION  ####

userid = 253889

x = userid
cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
randomframe = pd.DataFrame()
randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
if len(randomframe) ==0:
    raise Exception("No captions in Winkl database")
r2 = pd.DataFrame()
r2 = prepro(randomframe,'Caption')
# Merge
r3 = pd.DataFrame()
r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
caption_array = r3['Caption'][x]
# to 1D
caption_array = flat(caption_array)
# Word2vec computation
frame = pd.DataFrame()
frame = pd.DataFrame(compute(caption_array,categories,3))
display(frame)
frame = pd.DataFrame(compute_UC(caption_array,categories,3))
display(frame)



C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,verified,smile,killing,category,Scores
0,0,0,1,food,1
1,0,0,0,fashion,0
2,0,1,0,makeup,1
3,0,1,1,beauty,2
4,0,0,0,lifestyle,0
5,0,0,0,luxury,0
6,13,0,0,traveler,13
7,0,0,0,art,0
8,0,1,0,photography,1
9,0,0,0,fitness,0


,verified,smile,killing
0,0.054156,0.050950,0.096773
1,-0.053053,0.087240,0.021573
2,0.006139,0.178881,0.052030
3,-0.004304,0.188484,0.090230
4,0.012644,0.096752,0.046932
5,0.019637,0.026981,0.018293
6,0.087663,0.067354,0.015066
7,0.016567,0.066963,-0.013470
8,0.009623,0.134366,0.076769
9,0.053616,0.087908,0.006452


In [32]:
testarr = frame['one'].tolist()
print(testarr)
sdarr = deviation(testarr)
print(sdarr)
fff= dev_shortlist(sdarr, thre = 3)
print(fff)

[0.0888483077287674, 0.14774522185325623, 0.11855874955654144, 0.12943017482757568, 0.04335587099194527, 0.16439156234264374, 0.12480656057596207, 0.07402688264846802, 0.050342101603746414, 0.05671370401978493, 0.16844531893730164, 0.0973549336194992, 0.11009986698627472, 0.060157448053359985, 0.055308349430561066, 0.04075391963124275, 0.09393852949142456]
[6.422776241788562, 1.6703151253481825, 4.025405825397745, 3.1481777626222915, 10.09361434761687, 0.327102376720533, 3.5212626224125416, 7.618734424512254, 9.529887201184499, 9.01575512513656, 0.0, 5.736366616110082, 4.7079629865209425, 8.737875377821682, 9.129154836129574, 10.303568856311006, 6.012040274366657]
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


In [117]:

w.similarity('map','delhi')

0.038963214